In [35]:
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


mot

In [1]:
import pandas as pd
import numpy as np

cities = [
    "Bac Lieu", "Ho Chi Minh City", "Tam Ky", "Ben Tre", "Hoa Binh", "Tan An", 
    "Bien Hoa", "Hong Gai", "Thai Nguyen", "Buon Me Thuot", "Buon Ma Thuot", "Hue", "Thanh Hoa", 
    "Ca Mau", "Long Xuyen", "Tra Vinh", "Cam Pha", "My Tho", "Tuy Hoa", 
    "Cam Ranh", "Nam Dinh", "Uong Bi", "Can Tho", "Nha Trang", "Viet Tri", 
    "Chau Doc", "Phan Rang", "Vinh", "Da Lat", "Phan Thiet", "Vinh Long", 
    "Ha Noi", "Play Cu", "Pleiku", "Vung Tau", "Hai Duong", "Qui Nhon", "Yen Bai", 
    "Hai Phong", "Rach Gia", "Hanoi", "Soc Trang"
]

top_cities = [ 
    "Hanoi", "Ha Noi", "Hai Phong", "Vinh", "Hue", "Nha Trang",
    "Da Lat", "Buon Ma Thuot", "Ho Chi Minh City", "Can Tho", "Ca Mau"
]

In [9]:
import pandas as pd

top_cities = [ 
    "Hanoi", "Ha Noi", "Hai Phong", "Vinh", "Hue", "Nha Trang",
    "Da Lat", "Buon Me Thuot", "Buon Ma Thuot", "Ho Chi Minh City", "Can Tho", "Ca Mau"
]

# 1. Read the file
df2 = pd.read_csv("vietnam_weather_kaggle.csv")

# 2. Filter: Keep rows where 'city' IS IN the list
# This keeps "Ha Noi" OR "Hanoi" OR "Hue", etc.
df_filtered = df2[df2['province'].isin(top_cities)]

# 3. Save
df_filtered.to_csv("vietnam_weather_kaggle.csv", index=False)
print(f"✅ Reduced from {len(df2)} to {len(df_filtered)} rows.")

✅ Reduced from 45490 to 45490 rows.


In [10]:
import pandas as pd

# 1. Load the two files
# We use 'index_col=None' to ensure we read everything raw
df1 = pd.read_csv("vietnam_weather_daily.csv")       # Your first file
df2 = pd.read_csv("vietnam_weather_daily2.csv")       # The new file you just grabbed

# 2. Check for "duplicate index columns"
# Your snippet shows file 1 has a "0, 1, 2" column at the start. 
# Sometimes this gets named "Unnamed: 0". We should remove it if it exists.
if 'Unnamed: 0' in df1.columns:
    df1 = df1.drop(columns=['Unnamed: 0'])

if 'Unnamed: 0' in df2.columns:
    df2 = df2.drop(columns=['Unnamed: 0'])

# 3. Stack them together
# ignore_index=True resets the row numbers to 0, 1, 2, 3... all the way to the end
full_df = pd.concat([df1, df2], ignore_index=True)

# 4. Remove duplicates (Optional but recommended)
# Just in case you accidentally grabbed "Hanoi" in both files
full_df = full_df.drop_duplicates()

# 5. Save the final result
full_df.to_csv("vietnam_weather_complete.csv", index=False)

print(f"✅ Merged! Total rows: {len(full_df)}")
print(full_df.head())

✅ Merged! Total rows: 45520
         time  temperature_2m_max  temperature_2m_min  temperature_2m_mean  \
0  2009-01-01                17.3                13.4                 15.1   
1  2009-01-02                17.6                12.2                 14.8   
2  2009-01-03                18.3                10.8                 14.4   
3  2009-01-04                20.6                12.4                 16.2   
4  2009-01-05                22.6                14.1                 17.6   

   precipitation_sum rain_sum wind_speed_10m_max   city official_name  
0                0.0      0.0               11.2  Hanoi         Hanoi  
1                0.0      0.0                9.7  Hanoi         Hanoi  
2                0.0      0.0                8.1  Hanoi         Hanoi  
3                0.0      0.0               10.5  Hanoi         Hanoi  
4                0.1      0.1               11.1  Hanoi         Hanoi  


In [14]:
import pandas as pd

# 1. Load the Datasets
df_om = pd.read_csv("vietnam_weather_complete.csv")       # Your Master Data (Open-Meteo)
df_kag = pd.read_csv("vietnam_weather_kaggle.csv")     # The source for extra columns

# 2. Standardize Dates (Crucial for matching)
# Open-Meteo is usually YYYY-MM-DD. Kaggle is usually DD/MM/YYYY.
df_om["time"] = pd.to_datetime(df_om["time"])
df_kag["date"] = pd.to_datetime(df_kag["date"], dayfirst=True)

# 3. Standardize City Names
# We need them to be IDENTICAL. "Hanoi" != "Ha Noi" to a computer.
# Let's clean the Kaggle names to match your Open-Meteo names.

city_map = {
    "Ha Noi": "Hanoi",            # If Kaggle says Ha Noi, change to Hanoi
    "Thua Thien Hue": "Hue",      # If Kaggle uses full province name
    "TP. Ho Chi Minh": "Ho Chi Minh City",
    "Buon Me Thuot": "Buon Ma Thuot",
    # Add other mismatches here if you see empty results later
}

df_kag["province"] = df_kag["province"].replace(city_map)

# 4. Prepare the Kaggle Data for Merging
# We ONLY want the new columns, plus the keys (city, date)
cols_to_add = ["province", "date", "humidi", "cloud", "pressure"]
df_kag_clean = df_kag[cols_to_add].copy()

# Rename Kaggle columns to match Open-Meteo keys
df_kag_clean = df_kag_clean.rename(columns={
    "province": "city",
    "date": "time",
    "humidi": "humidity_avg",   
    "cloud": "cloud_cover_avg",
    "pressure": "pressure_avg"
})

# 5. Perform the Merge (Left Join)
merged_df = pd.merge(df_om, df_kag_clean, on=["city", "time"], how="left")

# 6. Check results
print(f"Total rows: {len(merged_df)}") 
print("Sample of rows with matches:")
# Show rows where we actually found humidity data
print(merged_df[merged_df['humidity_avg'].notnull()].head())

# 7. Save
merged_df.to_csv("vietnam_weather_final.csv", index=False)
print("✅ Saved to 'vietnam_weather_final.csv'")

Total rows: 45520
Sample of rows with matches:
        time  temperature_2m_max  temperature_2m_min  temperature_2m_mean  \
0 2009-01-01                17.3                13.4                 15.1   
1 2009-01-02                17.6                12.2                 14.8   
2 2009-01-03                18.3                10.8                 14.4   
3 2009-01-04                20.6                12.4                 16.2   
4 2009-01-05                22.6                14.1                 17.6   

   precipitation_sum rain_sum wind_speed_10m_max   city official_name  \
0                0.0      0.0               11.2  Hanoi         Hanoi   
1                0.0      0.0                9.7  Hanoi         Hanoi   
2                0.0      0.0                8.1  Hanoi         Hanoi   
3                0.0      0.0               10.5  Hanoi         Hanoi   
4                0.1      0.1               11.1  Hanoi         Hanoi   

   humidity_avg  cloud_cover_avg  pressure_avg  
0 

In [15]:
import pandas as pd

# 1. Load your data
df = pd.read_csv("vietnam_weather_final.csv")

# 2. Filter: Keep rows where city is NOT "Ha Noi"
df = df[df['city'] != "Ha Noi"]

# Optional: Reset the index so it flows 0, 1, 2, 3... nicely
df = df.reset_index(drop=True)

# 3. Save
df.to_csv("vietnam_weather_final.csv", index=False)

print(f"✅ Dropped all 'Ha Noi' rows. Remaining rows: {len(df)}")

✅ Dropped all 'Ha Noi' rows. Remaining rows: 40968
